# Preprocessing

In [1]:
import pandas as pd 
import numpy as np

In [2]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"

In [3]:
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [4]:
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

In [5]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [6]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [7]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [8]:
basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


In [9]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9324314 entries, 0 to 9324313
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 640.2+ MB


In [10]:
basics.describe()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,9324314,9324314,9324303,9324303,9324314,9324314,9324314,9324314,9324304
unique,9324314,11,4270101,4290758,9,151,96,878,2330
top,tt0000001,tvEpisode,Episode #1.1,Episode #1.1,0,\N,\N,\N,Drama
freq,1,7036854,45587,45587,9034717,1248377,9226557,6782939,1058664


In [11]:
basics.replace({'\\N':np.nan}, inplace = True)

In [12]:
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])

In [13]:
# remove all but 'movie'
filter = basics['titleType'] == 'movie'
basics= basics.loc[filter]

In [14]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [15]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282059 entries, 8 to 9324163
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          282059 non-null  object
 1   titleType       282059 non-null  object
 2   primaryTitle    282059 non-null  object
 3   originalTitle   282059 non-null  object
 4   isAdult         282059 non-null  object
 5   startYear       277689 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  282059 non-null  object
 8   genres          282059 non-null  object
dtypes: object(9)
memory usage: 21.5+ MB


In [16]:
basics['startYear'] = basics['startYear'].astype(float)

In [17]:
# only keeping 2000-2002
filter2 = (basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)
basics = basics[filter2]

In [18]:
akas.replace({'\\N':np.nan}, inplace = True)

In [19]:
akas.dropna(subset=['region'], inplace=True)

In [20]:
akas.isna().sum()

titleId                   0
ordering                  0
title                     3
region                    0
language            4457551
types              28226164
attributes         31510742
isOriginalTitle        2175
dtype: int64

In [23]:
only_US = akas['region'].str.contains('US',case=False)
akas = akas[only_US]

In [24]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [25]:
ratings.replace({'\\N':np.nan},inplace = True)

In [26]:
keepers_ratings =ratings['tconst'].isin(akas['titleId'])
keepers_ratings

0           True
1           True
2          False
3          False
4           True
           ...  
1240688    False
1240689     True
1240690    False
1240691    False
1240692    False
Name: tconst, Length: 1240693, dtype: bool

In [27]:
ratings = ratings[keepers_ratings]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1920
1,tt0000002,5.8,260
4,tt0000005,6.2,2541
5,tt0000006,5.1,175
6,tt0000007,5.4,797
...,...,...,...
1240667,tt9916204,8.2,245
1240673,tt9916348,8.5,17
1240674,tt9916362,6.4,4935
1240678,tt9916428,3.8,14


In [28]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34793       True
61095       True
67641       True
77935      False
86771       True
           ...  
9323986     True
9323995     True
9324034    False
9324079     True
9324163    False
Name: tconst, Length: 144267, dtype: bool

In [29]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34793,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61095,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
67641,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93907,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
9323450,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
9323846,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9323986,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
9323995,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [30]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83107 entries, 34793 to 9324079
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          83107 non-null  object 
 1   titleType       83107 non-null  object 
 2   primaryTitle    83107 non-null  object 
 3   originalTitle   83107 non-null  object 
 4   isAdult         83107 non-null  object 
 5   startYear       83107 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  83107 non-null  object 
 8   genres          83107 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.3+ MB


In [31]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1362165 entries, 5 to 33615809
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1362165 non-null  object
 1   ordering         1362165 non-null  int64 
 2   title            1362165 non-null  object
 3   region           1362165 non-null  object
 4   language         3702 non-null     object
 5   types            965157 non-null   object
 6   attributes       45232 non-null    object
 7   isOriginalTitle  1360790 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.5+ MB


In [32]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472456 entries, 0 to 1240689
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         472456 non-null  object 
 1   averageRating  472456 non-null  float64
 2   numVotes       472456 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.4+ MB


## Saving Data

In [33]:
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints',
 'title.akas.tsv.gz',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [34]:
# Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [35]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [36]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [37]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [38]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [39]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1920
1,tt0000002,5.8,260
2,tt0000005,6.2,2541
3,tt0000006,5.1,175
4,tt0000007,5.4,797
